In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()

        # First conv layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2, padding_mode='reflect')

        # Second conv layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)  # 5x5 is the size after two convs and poolings
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        # conv+relu+maxpool
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # conv+relu+maxpool
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # flatten tensor
        x = x.view(-1, 16 * 5 * 5)
        
        # Fully connected layers + ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # output
        x = self.fc3(x)
        return x


In [3]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
#from dl.models import LeNet5

# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 40

# Dataset preparation
transform = transforms.ToTensor()

# Download and load QMNIST dataset
train_dataset = torchvision.datasets.QMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.QMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the model
model = LeNet5()

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training function
def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')
        
# Testing function
def test_model(model, test_loader, criterion):
    model.eval()  # Evaluation mode
    correct = 0
    total = 0
    test_loss = 0.0
    
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # Get predicted class
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    print(f'Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Main training loop
if __name__ == "__main__":
    # Train the model
    train_model(model, train_loader, optimizer, criterion, epochs)
    
    # Test the model
    test_model(model, test_loader, criterion)
    
    # Save the model for later reuse
    torch.save(model.state_dict(), 'lenet5_qmnist.pth')
    print('Model saved as lenet5_qmnist.pth')


Epoch [1/40], Loss: 0.2677
Epoch [2/40], Loss: 0.0692
Epoch [3/40], Loss: 0.0491
Epoch [4/40], Loss: 0.0402
Epoch [5/40], Loss: 0.0318
Epoch [6/40], Loss: 0.0281
Epoch [7/40], Loss: 0.0225
Epoch [8/40], Loss: 0.0205
Epoch [9/40], Loss: 0.0171
Epoch [10/40], Loss: 0.0153
Epoch [11/40], Loss: 0.0133
Epoch [12/40], Loss: 0.0116
Epoch [13/40], Loss: 0.0107
Epoch [14/40], Loss: 0.0091
Epoch [15/40], Loss: 0.0088
Epoch [16/40], Loss: 0.0084
Epoch [17/40], Loss: 0.0079
Epoch [18/40], Loss: 0.0069
Epoch [19/40], Loss: 0.0061
Epoch [20/40], Loss: 0.0075
Epoch [21/40], Loss: 0.0053
Epoch [22/40], Loss: 0.0057
Epoch [23/40], Loss: 0.0059
Epoch [24/40], Loss: 0.0049
Epoch [25/40], Loss: 0.0060
Epoch [26/40], Loss: 0.0051
Epoch [27/40], Loss: 0.0049
Epoch [28/40], Loss: 0.0047
Epoch [29/40], Loss: 0.0032
Epoch [30/40], Loss: 0.0051
Epoch [31/40], Loss: 0.0039
Epoch [32/40], Loss: 0.0044
Epoch [33/40], Loss: 0.0050
Epoch [34/40], Loss: 0.0034
Epoch [35/40], Loss: 0.0030
Epoch [36/40], Loss: 0.0051
E